In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'

In [ ]:
# stdlib
import os

# third party
import numpy as np
import requests

# syft absolute
import syft as sy

from getpass import getpass  # noqa


sy.requires(SYFT_VERSION)

# syft absolute
from syft.service.worker.image_registry import SyftImageRegistry
from syft.service.worker.worker_image import SyftWorkerImage

In [ ]:
os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
os.environ["DEV_MODE"] = "True"

# Uncomment this to add custom values
# os.environ["NODE_URL"] = "http://localhost"
# os.environ["NODE_PORT"] = "8080"

In [ ]:
domain = sy.orchestra.launch(
    name="test-domain-1",
    dev_mode=True,
)

In [ ]:
domain_client = domain.login(email="info@openmined.org", password="changethis")
domain_client

### Scaling Default Worker Pool

We should see a default worker pool

In [ ]:
domain_client.worker_pools

Scale up to 3 workers

In [ ]:
result = domain_client.api.services.worker_pool.scale(
    number=3, pool_name="default-pool"
)
assert not isinstance(result, sy.SyftError), str(result)
result

In [ ]:
result = domain_client.api.services.worker_pool.get_by_name(pool_name="default-pool")
assert len(result.workers) == 3, str(result.to_dict())
result

In [ ]:
# stdlib
# wait for some time for scale up to be ready
from time import sleep

sleep(5)

Scale down to 1 worker

In [ ]:
default_pool_scale_res = domain_client.api.services.worker_pool.scale(
    number=1, pool_name="default-pool"
)
assert not isinstance(default_pool_scale_res, sy.SyftError), str(default_pool_scale_res)
default_pool_scale_res

In [ ]:
result = domain_client.api.services.worker_pool.get_by_name(pool_name="default-pool")
assert len(result.workers) == 1, str(result.to_dict())
result

In [ ]:
default_worker_pool = domain_client.api.services.worker_pool.get_by_name(
    pool_name="default-pool"
)
default_worker_pool

#### Submit Dockerfile

In [ ]:
registry = os.getenv("SYFT_BASE_IMAGE_REGISTRY", "docker.io")
repo = "openmined/grid-backend"

if "k3d" in registry:
    res = requests.get(url=f"http://{registry}/v2/{repo}/tags/list")
    tag = res.json()["tags"][0]
else:
    tag = sy.__version__

In [ ]:
custom_dockerfile_str = f"""
FROM {registry}/{repo}:{tag}

RUN pip install pydicom

""".strip()

In [ ]:
docker_config = sy.DockerWorkerConfig(dockerfile=custom_dockerfile_str)

In [ ]:
assert docker_config.dockerfile == custom_dockerfile_str

In [ ]:
submit_result = domain_client.api.services.worker_image.submit_dockerfile(
    docker_config=docker_config
)
submit_result

In [ ]:
assert isinstance(submit_result, sy.SyftSuccess), str(submit_result)

In [ ]:
dockerfile_list = domain_client.images.get_all()
dockerfile_list

In [ ]:
assert not isinstance(dockerfile_list, sy.SyftError), str(dockerfile_list)
assert len(dockerfile_list) == 2

In [ ]:
workerimage = next(
    (
        image
        for image in dockerfile_list
        if not image.is_prebuilt and image.config.dockerfile == custom_dockerfile_str
    ),
    None,
)

assert isinstance(workerimage, SyftWorkerImage), str(workerimage)
workerimage

#### Add External Registry in Syft

In [ ]:
external_registry = os.getenv("EXTERNAL_REGISTRY", registry)
external_registry_username = os.getenv("EXTERNAL_REGISTRY_USERNAME", None)
external_registry_password = os.getenv("EXTERNAL_REGISTRY_PASSWORD", None)

# external_registry = input()
# external_registry_username = getpass("Enter Registry Username")
# external_registry_password = getpass("Enter Registry Password")

In [ ]:
registry_add_result = domain_client.api.services.image_registry.add(external_registry)
registry_add_result

In [ ]:
assert isinstance(registry_add_result, sy.SyftSuccess), str(registry_add_result)

In [ ]:
image_registry_list = domain_client.api.services.image_registry.get_all()
image_registry_list

In [ ]:
assert not isinstance(image_registry_list, sy.SyftError), str(image_registry_list)
assert len(image_registry_list) == 1

In [ ]:
local_registry = image_registry_list[0]
local_registry

In [ ]:
assert isinstance(local_registry, SyftImageRegistry), str(local_registry)

In [ ]:
registry_uid = local_registry.id

#### Build Image

In [ ]:
docker_tag = "openmined/custom-worker:0.7.8"


docker_build_result = domain_client.api.services.worker_image.build(
    image_uid=workerimage.id,
    tag=docker_tag,
    registry_uid=registry_uid,
)
docker_build_result

In [ ]:
assert not isinstance(docker_build_result, sy.SyftError), str(docker_build_result)

In [ ]:
image_list = domain_client.images.get_all()
image_list

In [ ]:
# we can also index with string using the repo_with_tag format
workerimage = next((image for image in image_list if image.id == workerimage.id), None)
workerimage

In [ ]:
assert workerimage is not None, str([image.__dict__ for image in image_list])
assert workerimage.is_built is not None, str(workerimage)
assert workerimage.built_at is not None, str(workerimage)
assert workerimage.image_hash is not None, str(workerimage)
assert image_list[workerimage.built_image_tag] == workerimage

#### Push Image to Local Registry

In [ ]:
push_result = None
push_result = domain_client.api.services.worker_image.push(
    workerimage.id,
    username=external_registry_username,
    password=external_registry_password,
)
push_result

In [ ]:
assert isinstance(push_result, sy.SyftSuccess), str(push_result)

In [ ]:
base_url = f"http://{workerimage.image_identifier.registry_host}"
expected_tag = workerimage.image_identifier.tag

repos = requests.get(f"{base_url}/v2/_catalog").json()["repositories"]
tags = requests.get(f"{base_url}/v2/openmined/custom-worker/tags/list").json()
tags = tags["tags"]

assert (
    "openmined/custom-worker" in repos
), f"'openmined/custom-worker' not uploaded to local registry | {repos}"
assert (
    expected_tag in tags
), f"'openmined/custom-worker' with tag {expected_tag} not available | {tags}"

#### Create Worker Pool From Image

In [ ]:
worker_pool_name = "custom-pool"
worker_pool_res = domain_client.api.services.worker_pool.launch(
    name=worker_pool_name,
    image_uid=workerimage.id,
    num_workers=3,
    reg_username=external_registry_username,
    reg_password=external_registry_password,
)

In [ ]:
assert not isinstance(worker_pool_res, sy.SyftError), str(worker_pool_res)
assert len(worker_pool_res) == 3

In [ ]:
for status in worker_pool_res:
    assert status.error is None

In [ ]:
worker_pool_list = domain_client.worker_pools.get_all()
worker_pool_list

In [ ]:
assert not isinstance(worker_pool_list, sy.SyftError), str(worker_pool_res)
assert len(worker_pool_list) == 2

In [ ]:
worker_pool = next(
    (pool for pool in worker_pool_list if pool.name == worker_pool_name),
    None,
)

assert worker_pool is not None, str(
    [worker_pool.__dict__ for worker_pool in worker_pool_list]
)
assert len(worker_pool.workers) == 3

In [ ]:
# We can filter pools based on the image id upon which the pools were built
filtered_result = domain_client.api.services.worker_pool.filter_by_image_id(
    image_uid=workerimage.id
)
filtered_result

In [ ]:
assert not isinstance(filtered_result, sy.SyftError), str(filtered_result)

In [ ]:
second_worker = worker_pool.workers[1]
second_worker

#### Get Worker Logs

In [ ]:
worker_logs = domain_client.api.services.worker.logs(
    uid=second_worker.id,
)
worker_logs

In [ ]:
assert isinstance(worker_logs, str)

In [ ]:
worker_pool

### Syft function

In [ ]:
data = np.array([1, 2, 3])
data_action_obj = sy.ActionObject.from_obj(data)

data_pointer = domain_client.api.services.action.set(data_action_obj)
data_pointer

In [ ]:
@sy.syft_function(
    input_policy=sy.ExactMatch(x=data_pointer),
    output_policy=sy.SingleExecutionExactOutput(),
    worker_pool_name=worker_pool_name,
)
def custom_worker_func(x):
    return {"y": x + 1}

In [ ]:
custom_worker_func

In [ ]:
assert custom_worker_func.worker_pool_name == worker_pool.name

In [ ]:
request = domain_client.code.request_code_execution(custom_worker_func)
request

In [ ]:
domain_client.requests[-1].approve(approve_nested=True)

In [ ]:
job = domain_client.code.custom_worker_func(x=data_pointer, blocking=False)
job

In [ ]:
worker_pool = domain_client.worker_pools[worker_pool_name]
worker_pool

In [ ]:
job.wait()

In [ ]:
assert job.status.value == "completed"

In [ ]:
job_list = domain_client.jobs.get_by_user_code_id(job.user_code_id)

In [ ]:
assert not isinstance(job_list, sy.SyftError), job_list

In [ ]:
job_refresh = job_list[0]
assert job_refresh.job_worker_id is not None, str([job.to_dict() for job in job_list])

In [ ]:
# Validate the result received from the syft function
result = job.wait().get()
result_matches = result["y"] == data + 1
assert result_matches.all()

In [ ]:
# Scale Down the workers
custom_pool_scale_res = domain_client.api.services.worker_pool.scale(
    number=1, pool_name=worker_pool_name
)
assert not isinstance(custom_pool_scale_res, sy.SyftError), str(custom_pool_scale_res)
custom_pool_scale_res

In [ ]:
assert len(domain_client.worker_pools[worker_pool_name].worker_list) == 1

#### Worker Pool and Image Creation Request/Approval

In [ ]:
dockerfile_opendp = f"""
FROM {registry}/{repo}:{tag}

RUN pip install opendp
""".strip()

docker_config_opendp = sy.DockerWorkerConfig(dockerfile=dockerfile_opendp)

In [ ]:
submit_result = None
submit_result = domain_client.api.services.worker_image.submit_dockerfile(
    docker_config=docker_config_opendp
)
submit_result

In [ ]:
assert isinstance(submit_result, sy.SyftSuccess), str(submit_result)

In [ ]:
_images = domain_client.images
assert not isinstance(_images, sy.SyftError), str(_images)

In [ ]:
workerimage_opendp = next(
    (im for im in _images if im.config == docker_config_opendp),
    None,
)
assert workerimage_opendp is not None, str([im.__dict__ for im in _images])

##### Build image first then create pool

In [ ]:
docker_tag_opendp = "openmined/custom-worker-opendp:latest"

docker_build_result = domain_client.api.services.worker_image.build(
    image_uid=workerimage_opendp.id,
    tag=docker_tag_opendp,
    registry_uid=registry_uid,
)

docker_build_result

In [ ]:
assert isinstance(docker_build_result, sy.SyftSuccess), str(docker_build_result)

In [ ]:
_images = domain_client.images
assert not isinstance(_images, sy.SyftError), str(_images)

In [ ]:
workerimage_opendp = next(
    (image for image in _images if image.id == workerimage_opendp.id),
    None,
)
assert workerimage_opendp is not None, str([image.__dict__ for image in _images])
assert workerimage_opendp.is_built is not None, str(workerimage_opendp.__dict__)
assert workerimage_opendp.built_at is not None, str(workerimage_opendp.__dict__)
assert workerimage_opendp.image_hash is not None, str(workerimage_opendp.__dict__)

assert _images[workerimage_opendp.built_image_tag] == workerimage_opendp, str(
    workerimage_opendp
)

workerimage_opendp

In [ ]:
# Push OpenDP Image to registry
push_result = None
push_result = domain_client.api.services.worker_image.push(
    workerimage_opendp.id,
    username=external_registry_username,
    password=external_registry_password,
)
assert isinstance(push_result, sy.SyftSuccess), str(push_result)

In [ ]:
pool_name_opendp = "opendp-pool"
pool_create_request = domain_client.api.services.worker_pool.pool_creation_request(
    pool_name=pool_name_opendp, num_workers=3, image_uid=workerimage_opendp.id
)
pool_create_request

In [ ]:
assert not isinstance(pool_create_request, sy.SyftError), str(pool_create_request)
assert len(pool_create_request.changes) == 1

In [ ]:
# get the pending request and approve it
req_result = pool_create_request.approve(
    reg_username=external_registry_username, reg_password=external_registry_password
)
req_result

In [ ]:
assert isinstance(req_result, sy.SyftSuccess), str(req_result)

In [ ]:
pool_opendp = domain_client.worker_pools[pool_name_opendp]
assert not isinstance(pool_opendp, sy.SyftError), str(pool_opendp)
assert len(pool_opendp.worker_list) == 3

In [ ]:
worker_pool_list = domain_client.worker_pools.get_all()

assert not isinstance(worker_pool_list, sy.SyftError), str(worker_pool_list)
assert len(worker_pool_list) == 3

In [ ]:
# Scale Down the workers
opendp_pool_scale_res = domain_client.api.services.worker_pool.scale(
    number=1, pool_name=pool_name_opendp
)
assert not isinstance(opendp_pool_scale_res, sy.SyftError), str(opendp_pool_scale_res)
opendp_pool_scale_res

In [ ]:
assert len(domain_client.worker_pools[pool_name_opendp].worker_list) == 1

Request to build the image and create the pool at the same time

In [ ]:
dockerfile_recordlinkage = f"""
FROM {registry}/{repo}:{tag}

RUN pip install recordlinkage
""".strip()

docker_config_recordlinkage = sy.DockerWorkerConfig(dockerfile=dockerfile_recordlinkage)

docker_tag_recordlinkage = "openmined/custom-worker-recordlinkage:latest"

In [ ]:
pool_name_recordlinkage = "recordlinkage-pool"

pool_image_create_request = (
    domain_client.api.services.worker_pool.create_image_and_pool_request(
        pool_name=pool_name_recordlinkage,
        num_workers=2,
        tag=docker_tag_recordlinkage,
        config=docker_config_recordlinkage,
        registry_uid=registry_uid,
        reason="I want to do some more cool data science with PySyft and OpenDP",
    )
)
pool_image_create_request

In [ ]:
assert not isinstance(pool_image_create_request, sy.SyftError), str(
    pool_image_create_request
)

In [ ]:
assert len(pool_image_create_request.changes) == 2
assert pool_image_create_request.changes[0].config == docker_config_recordlinkage
assert pool_image_create_request.changes[1].num_workers == 2
assert pool_image_create_request.changes[1].pool_name == pool_name_recordlinkage

In [ ]:
req_result = pool_image_create_request.approve(
    reg_username=external_registry_username, reg_password=external_registry_password
)
req_result

In [ ]:
assert isinstance(req_result, sy.SyftSuccess), str(req_result)

In [ ]:
_requests = domain_client.requests
assert not isinstance(_requests, sy.SyftError), str(_requests)

In [ ]:
pool_image_create_request = next(
    (req for req in _requests if req.id == pool_image_create_request.id),
    None,
)
assert pool_image_create_request is not None, str([req.__dict__ for req in _requests])
assert pool_image_create_request.status.value == 2, str(pool_image_create_request)

In [ ]:
domain_client.images

In [ ]:
image_exists = False
for im in domain_client.images.get_all():
    if (
        im.image_identifier
        and im.image_identifier.repo_with_tag == docker_tag_recordlinkage
    ):
        image_exists = True

assert image_exists, str([im.__dict__ for im in _images])

In [ ]:
assert domain_client.worker_pools[pool_name_recordlinkage]
assert len(domain_client.worker_pools[pool_name_recordlinkage].worker_list) == 2

In [ ]:
# Scale down the workers
recordlinkage_pool_scale_res = domain_client.api.services.worker_pool.scale(
    number=1, pool_name=pool_name_recordlinkage
)
assert not isinstance(recordlinkage_pool_scale_res, sy.SyftError), str(
    recordlinkage_pool_scale_res
)
recordlinkage_pool_scale_res

In [ ]:
assert len(domain_client.worker_pools[pool_name_recordlinkage].worker_list) == 1